In [ ]:
import csv
import requests
import math
import asyncio
import time
import itertools
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

import numpy as np
from matplotlib import pyplot as plt

from binance.client import Client

In [ ]:
# Binance 캔들 가져오기
def get_data(ticker, since, now, future=False):
    client = Client()
    if future:
        klines = client.futures_historical_klines(ticker, Client.KLINE_INTERVAL_1MINUTE, since, now)
    else:
        klines = client.get_historical_klines(ticker, Client.KLINE_INTERVAL_1MINUTE, since, now, 1000)

    csv_data = np.array(klines, dtype=np.float64)
    print("get_data: ", len(csv_data))

    return csv_data

In [ ]:
# 최근 데이터만 가져와 저장
X_len = 128
time_windows = [1]
for i in range(2,17):
    time_windows.append(i)

now = Client().get_server_time()['serverTime']
#since = now - 1000 * 60 * ((X_len-1)*time_windows[-1] + 1 + 1440*7)
since = int((time.mktime(datetime.strptime("2022-08-20 07:07:08", "%Y-%m-%d %H:%M:%S").timetuple()) * 1000) + (1000  * 60 * 60 * 9) - (1000 * 60 * ((X_len-1)*time_windows[-1] + 1)))

print("since: ", datetime.fromtimestamp(since / 1000))
print("now: ", datetime.fromtimestamp(now / 1000))

symbols = ["DOGEUSDT"]
for symbol in symbols:
    ticker = symbol.split("USDT")[0]
    print(ticker)

    csv_data = get_data(symbol, since, now, future=True)
    np.save("../Data/future/usdt/test_recent/" + ticker + "_minute.npy", csv_data)

In [ ]:
# 훈련, 테스트 데이터 기간 설정
train_end = datetime(2021,1,1,0,0)
train_end = int(time.mktime(train_end.timetuple()) * 1000)
print("train_end: ", end='')
print(train_end)
test_end = datetime(2022,6,1,0,0)
test_end = int(time.mktime(test_end.timetuple()) * 1000)
print("test_end: ", end='')
print(test_end)

In [ ]:
# 훈련 데이터가 존재하는 종목 확인
def get_ticker(train_end, future=False):
    client = Client()
    if future:
        markets = client.futures_ticker()
    else:
        markets = client.get_ticker()

    not_contains = ['UP', 'DOWN', 'USD', 'AUD', 'BIDR', 'BRL', 'EUR', 'GBP', 'RUB', 'TRY', 'TUSD', 'USDC', 'DAI', 'IDRT', 'UAH', 'NGN', 'VAI', 'USDP', 'BKRW', 'BZRX', 'BVND', 'UST']

    ticker_volumes = {}
    for market in markets:
        symbol = market['symbol']
        if symbol[-4:] == "USDT":
            ticker_valid = True
            ticker = symbol[:-4]
            if all([not_contain not in ticker for not_contain in not_contains]):
                if future:
                    ohlcvs = client.futures_historical_klines(symbol, Client.KLINE_INTERVAL_1DAY, 0)[0]
                else:
                    ohlcvs = client.get_historical_klines(symbol, Client.KLINE_INTERVAL_1DAY, 0)[0]
                if ohlcvs[0] <= train_end:
                    ticker_volumes[symbol] = float(market['quoteVolume'])

    sorted_ticker_volumes = sorted(ticker_volumes.items(), key=lambda item: item[1], reverse=True)
    symbols = np.array(sorted_ticker_volumes)[0:,0]
    
    return symbols

symbols = get_ticker(train_end, future=False)
print(len(symbols))
print(symbols)

In [ ]:
# USDT spot ticker
symbols = ['BTCUSDT', 'ETHUSDT', 'SOLUSDT', 'BNBUSDT', 'YFIIUSDT', 'XRPUSDT', 'MATICUSDT', 'AVAXUSDT', 'ADAUSDT', 'NEARUSDT', 'YFIUSDT', 'FTMUSDT', 'DOGEUSDT', 'RUNEUSDT', 'SANDUSDT', 'AXSUSDT', 'OGUSDT', 'WAVESUSDT', 'TRXUSDT', 'ZILUSDT', 'LINKUSDT', 'DOTUSDT', 'BCHUSDT', 'CELRUSDT', 'LTCUSDT', 'MANAUSDT', 'WNXMUSDT', 'ONEUSDT', 'AAVEUSDT', 'ATOMUSDT', 'THETAUSDT', 'VETUSDT', 'UNIUSDT', 'RSRUSDT', 'FILUSDT', 'BELUSDT', 'EGLDUSDT', 'LUNAUSDT', 'STORJUSDT', 'ROSEUSDT', 'SNXUSDT', 'ARPAUSDT', 'ETCUSDT', 'UNFIUSDT', 'ALGOUSDT', 'HNTUSDT', 'DUSKUSDT', 'CRVUSDT', 'GRTUSDT', 'OGNUSDT', 'COMPUSDT', 'FTTUSDT', 'UMAUSDT', 'LRCUSDT', 'SUSHIUSDT', 'BLZUSDT', 'BATUSDT', 'XMRUSDT', 'EOSUSDT', 'TRBUSDT', 'KAVAUSDT', 'ASRUSDT', 'XLMUSDT', '1INCHUSDT', 'CHZUSDT', 'SUNUSDT', 'KNCUSDT', 'BALUSDT', 'JSTUSDT', 'ZECUSDT', 'ENJUSDT', 'HBARUSDT', 'FLMUSDT', 'WINUSDT', 'ANTUSDT', 'XTZUSDT', 'CHRUSDT', 'FETUSDT', 'IOTXUSDT', 'GTOUSDT', 'IOSTUSDT', 'MKRUSDT', 'ALPHAUSDT', 'SXPUSDT', 'HOTUSDT', 'NEOUSDT', 'OMGUSDT', 'DASHUSDT', 'OCEANUSDT', 'REPUSDT', 'ANKRUSDT', 'KSMUSDT', 'BTSUSDT', 'BANDUSDT', 'SRMUSDT', 'CTSIUSDT', 'REEFUSDT', 'COTIUSDT', 'RENUSDT', 'RVNUSDT', 'DENTUSDT', 'QTUMUSDT', 'MTLUSDT', 'SKLUSDT', 'IOTAUSDT', 'TFUELUSDT', 'MBLUSDT', 'ZRXUSDT', 'PAXGUSDT', 'TCTUSDT', 'MDTUSDT', 'COCOSUSDT', 'INJUSDT', 'COSUSDT', 'BNTUSDT', 'ONTUSDT', 'JUVUSDT', 'WINGUSDT', 'ATMUSDT', 'DNTUSDT', 'AKROUSDT', 'CVCUSDT', 'XEMUSDT', 'ZENUSDT', 'ICXUSDT', 'TROYUSDT', 'ORNUSDT', 'WRXUSDT', 'NKNUSDT', 'CTKUSDT', 'XVSUSDT', 'TOMOUSDT', 'MFTUSDT', 'PSGUSDT', 'CTXCUSDT', 'NBSUSDT', 'MITHUSDT', 'KEYUSDT', 'VTHOUSDT', 'PNTUSDT', 'DGBUSDT', 'UTKUSDT', 'DREPUSDT', 'AIONUSDT', 'RLCUSDT', 'OXTUSDT', 'PERLUSDT', 'STXUSDT', 'NULSUSDT', 'HARDUSDT', 'LTOUSDT', 'SCUSDT', 'DIAUSDT', 'KMDUSDT', 'STPTUSDT', 'WTCUSDT', 'FIOUSDT', 'DATAUSDT', 'IRISUSDT', 'WANUSDT', 'BEAMUSDT', 'NMRUSDT', 'HIVEUSDT', 'STRAXUSDT', 'ONGUSDT', 'VITEUSDT', 'STMXUSDT', 'DOCKUSDT', 'AVAUSDT', 'LSKUSDT', 'DCRUSDT', 'ARDRUSDT', 'FUNUSDT', 'BCCUSDT', 'VENUSDT', 'PAXUSDT', 'BCHABCUSDT', 'BCHSVUSDT', 'BTTUSDT', 'NANOUSDT', 'ERDUSDT', 'NPXSUSDT', 'STORMUSDT', 'HCUSDT', 'MCOUSDT', 'BULLUSDT', 'BEARUSDT', 'ETHBULLUSDT', 'ETHBEARUSDT', 'EOSBULLUSDT', 'EOSBEARUSDT', 'XRPBULLUSDT', 'XRPBEARUSDT', 'STRATUSDT', 'BNBBULLUSDT', 'BNBBEARUSDT', 'XZCUSDT', 'GXSUSDT', 'LENDUSDT',]
print(len(symbols))

In [ ]:
# USDT future ticker
symbols = ['BTCUSDT', 'ETHUSDT', 'SOLUSDT', 'STORJUSDT', 'ADAUSDT', 'TRXUSDT', 'BNBUSDT', 'AVAXUSDT', 'LINKUSDT', 'XRPUSDT', 'FTMUSDT', 'DOTUSDT', 'NEARUSDT', 'WAVESUSDT', 'DOGEUSDT', 'PEOPLEUSDT', 'UNFIUSDT', 'GALAUSDT', 'SANDUSDT', 'LTCUSDT', 'AXSUSDT', 'EGLDUSDT', 'BCHUSDT', 'MANAUSDT', 'MATICUSDT', 'ATOMUSDT', 'THETAUSDT', '1000SHIBUSDT', 'ANTUSDT', 'HNTUSDT', 'XTZUSDT', 'BELUSDT', 'AAVEUSDT', 'ETCUSDT', 'ALICEUSDT', 'EOSUSDT', 'ZRXUSDT', 'OGNUSDT', 'XMRUSDT', 'RUNEUSDT', 'RSRUSDT', 'KNCUSDT', 'CRVUSDT', 'ZILUSDT', 'LITUSDT', 'UNIUSDT', 'BLZUSDT', 'RVNUSDT', 'ALGOUSDT', 'ATAUSDT', 'MTLUSDT', 'SFPUSDT', 'FILUSDT', 'GTCUSDT', 'DYDXUSDT', 'ENSUSDT', 'FLMUSDT', 'ICPUSDT', 'SUSHIUSDT', 'LRCUSDT', 'NKNUSDT', 'ZECUSDT', 'VETUSDT', 'BATUSDT', 'CHZUSDT', 'YFIUSDT', 'XLMUSDT', 'BANDUSDT', 'ONEUSDT', '1INCHUSDT', 'OCEANUSDT', 'CTKUSDT', 'COMPUSDT', 'SXPUSDT', 'ENJUSDT', 'STMXUSDT', 'C98USDT', 'ARUSDT', 'TLMUSDT', 'GRTUSDT', 'NEOUSDT', 'SRMUSDT', 'SCUSDT', 'TRBUSDT', 'BAKEUSDT', 'DASHUSDT', 'CELRUSDT', 'KSMUSDT', 'KAVAUSDT', 'SKLUSDT', 'LINAUSDT', 'MASKUSDT', 'CVCUSDT', 'ROSEUSDT', 'ALPHAUSDT', 'REEFUSDT', 'OMGUSDT', 'CHRUSDT', 'SNXUSDT', 'DENTUSDT', 'ICXUSDT', 'MKRUSDT', 'CELOUSDT', 'HOTUSDT', 'ANKRUSDT', 'HBARUSDT', 'IOSTUSDT', 'KLAYUSDT', 'QTUMUSDT', 'CTSIUSDT', 'RENUSDT', 'COTIUSDT', 'BALUSDT', 'IOTAUSDT', 'ONTUSDT', 'XEMUSDT', 'BTCDOMUSDT', 'ZENUSDT', 'LPTUSDT', 'TOMOUSDT', 'ARPAUSDT', 'RLCUSDT', 'IOTXUSDT', '1000XECUSDT', 'DEFIUSDT', 'RAYUSDT', 'BTSUSDT', 'DGBUSDT']
print(len(symbols))

In [ ]:
# 훈련, 테스트 데이터 가져와 저장
for i, symbol in enumerate(symbols):
    if i < 0: # 이 인덱스부터 시작
        continue
    #ticker = symbol.replace("USDT","")
    ticker = symbol.replace("USDT","")
    print(i, ticker)

    #csv_data = get_data(symbol, 0, train_end)
    #np.save("../Data/spot/usdt/train_new/" + ticker + "_minute.npy", csv_data)

    csv_data = get_data(symbol, train_end, test_end, future=True)
    np.save("../Data/future/usdt/test_22.05.31/" + ticker + "_minute.npy", csv_data)